In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [2]:
import zipfile
import os

# Unzip train and test data
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/train")

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/test")


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

2025-04-19 18:28:30.687543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745087310.867775      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745087310.923517      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Correct directory paths after unzipping
train_dir = "/kaggle/working/train/train"
test_dir = "/kaggle/working/test/test"

# Generate labels from filenames
filenames = os.listdir(train_dir)
labels = ['dog' if 'dog' in fname else 'cat' for fname in filenames]
df = pd.DataFrame({'filename': filenames, 'class': labels})

# Train-validation split (stratified to balance dog/cat distribution)
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['class'], random_state=42)

# Image settings
IMG_SIZE = 224
BATCH_SIZE = 32

# ImageDataGenerators
train_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

# Flow training data
train_data = train_gen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='filename',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

# Flow validation data
val_data = val_gen.flow_from_dataframe(
    dataframe=val_df,
    directory=train_dir,
    x_col='filename',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='binary',
    batch_size=BATCH_SIZE
)

# Prepare test data for prediction
test_filenames = os.listdir(test_dir)
test_df = pd.DataFrame({'filename': test_filenames})

test_data = test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    class_mode=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False  # Keep order for submission
)


Found 22500 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.
Found 12500 validated image filenames.


In [5]:
filenames = os.listdir(train_dir)
print(f"Number of training images found: {len(filenames)}")
print(f"First 5 files: {filenames[:5]}")


Number of training images found: 25000
First 5 files: ['cat.3405.jpg', 'dog.10176.jpg', 'cat.4818.jpg', 'cat.28.jpg', 'dog.4446.jpg']


In [6]:
#from tensorflow.keras.applications import MobileNetV2
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load base model without top layers
#base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#base_model.trainable = False  # Freeze base model

# Add custom layers on top
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dropout(0.3)(x)
#x = Dense(128, activation='relu')(x)
#output = Dense(1, activation='sigmoid')(x)  # Binary classification (dog vs cat)

#model = Model(inputs=base_model.input, outputs=output)
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#model.summary()


In [7]:
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
#checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

#history = model.fit(
    #train_data,
    #validation_data=val_data,
    #epochs=5,
    #callbacks=[early_stop, checkpoint]
#)


In [8]:
# Step 1: Predict on test set
#test_preds = model.predict(test_data)
#test_preds_labels = (test_preds > 0.5).astype(int).ravel()

# Step 2: Format the ID column correctly from filenames
#test_df['id'] = test_df['filename'].apply(lambda x: int(x.split('.')[0]))

# Step 3: Create submission dataframe
#submission = pd.DataFrame({
#    'id': test_df['id'],
#   'label': test_preds_labels
#}).sort_values('id')  # Sort to match Kaggle expected order

# Step 4: Save to CSV
#submission.to_csv('/kaggle/working/submission.csv', index=False)


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Unfreeze last 40 layers for fine-tuning
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-40]:
    layer.trainable = False
for layer in base_model.layers[-40:]:
    layer.trainable = True

# Build model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Increased dropout
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Stronger augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_gen = ImageDataGenerator(rescale=1./255)

# Rebuild data generators (if needed)
train_data = train_gen.flow_from_dataframe(
    train_df, train_dir, x_col='filename', y_col='class',
    target_size=(224, 224), class_mode='binary', batch_size=32
)

val_data = val_gen.flow_from_dataframe(
    val_df, train_dir, x_col='filename', y_col='class',
    target_size=(224, 224), class_mode='binary', batch_size=32
)

# Train model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


I0000 00:00:1745087324.238768      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 22500 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1745087344.356927      57 service.cc:148] XLA service 0x78ec7c152b10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745087344.357890      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745087345.860517      57 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1745087348.924873      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1745087349.122431      57 gpu_timer.cc:

  1/704 ━━━━━━━━━━━━━━━━━━━━ 5:04:10 26s/step - accuracy: 0.5312 - loss: 0.7663

I0000 00:00:1745087353.422248      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


704/704 ━━━━━━━━━━━━━━━━━━━━ 274s 352ms/step - accuracy: 0.9225 - loss: 0.1784 - val_accuracy: 0.9832 - val_loss: 0.0561 - learning_rate: 1.0000e-04
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 236s 332ms/step - accuracy: 0.9685 - loss: 0.0781 - val_accuracy: 0.9768 - val_loss: 0.0802 - learning_rate: 1.0000e-04
Epoch 3/5
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - accuracy: 0.9755 - loss: 0.0613
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
704/704 ━━━━━━━━━━━━━━━━━━━━ 236s 332ms/step - accuracy: 0.9755 - loss: 0.0613 - val_accuracy: 0.9844 - val_loss: 0.0656 - learning_rate: 1.0000e-04
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 239s 336ms/step - accuracy: 0.9815 - loss: 0.0527 - val_accuracy: 0.9840 - val_loss: 0.0470 - learning_rate: 2.0000e-05
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 238s 334ms/step - accuracy: 0.9822 - loss: 0.0433 - val_accuracy: 0.9840 - val_loss: 0.0408 - learning_rate: 2.0000e-05


In [10]:
# Predict on test set
test_preds = model.predict(test_data)
test_preds_labels = (test_preds > 0.5).astype(int).ravel()

# Create submission
test_df['id'] = test_df['filename'].apply(lambda x: int(x.split('.')[0]))
submission = pd.DataFrame({'id': test_df['id'], 'label': test_preds_labels})
submission = submission.sort_values('id')
submission.to_csv('/kaggle/working/submission.csv', index=False)


391/391 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step
